In [1]:
import pandas as pd
from sqlalchemy import create_engine
from geopandas import GeoDataFrame
import geopandas as gpd
from shapely.geometry import Point
import shapely.geometry as geom
import fiona
import collections
import json
import numpy as np
import glob
pd.set_option('display.max_columns', None)

In [ ]:
# grab hard skills, cities, qualifications tables

In [2]:
pwd

'/Users/michaelmainzer/Desktop/projects/publications/cyber/data'

In [48]:
files = glob.glob('./profiles/*.xls')
dfs = []
for file in files:
    df = pd.read_excel(file, sheet_name="Overview", header=None, skipfooter=1)
    df['MSA'] = file[29:-4]
    df.columns = ['Profiles','MSA']
    df = df[['MSA','Profiles']]
    dfs.append(df)

msaProfiles = pd.concat(dfs, ignore_index=True)

In [49]:
files = glob.glob('./profiles/*.xls')
dfs = []
for file in files:
    df = pd.read_excel(file, sheet_name="Top Hard Skills (2)",skiprows=2)
    df.rename(columns={'Profiles with Skill / Total Profiles (2017 - 2020)':'Profiles',
                       'Frequency in Profiles':'Frequency',
                       'Skill':'Metric'}, inplace=True)
    df[['Profiles','Totals']] = df['Profiles'].str.split('/',expand=True)
    df['MSA'] = file[29:-4]
    del df['Totals']
    del df['Frequency']
    dfs.append(df)

skills = pd.concat(dfs, ignore_index=True)

In [50]:
files = glob.glob('./profiles/*.xls')
dfs = []
for file in files:
    df = pd.read_excel(file, sheet_name="Top Qualifications (2)",skiprows=2)
    df.rename(columns={'Profiles with Qualification':'Profiles',
                       'Qualification':'Metric'}, inplace=True)
    df['MSA'] = file[29:-4]
    dfs.append(df)

qualifications = pd.concat(dfs, ignore_index=True)

In [51]:
files = glob.glob('./profiles/*.xls')
dfs = []
for file in files:
    df = pd.read_excel(file, sheet_name="Top Cities",skiprows=2)
    del df['Percent']
    df.rename(columns={'City':'Metric'}, inplace=True)
    df['MSA'] = file[29:-4]
    dfs.append(df)

cities = pd.concat(dfs, ignore_index=True)

In [52]:
skills['Category'] = "Skills"
qualifications['Category'] = "Qualifications"
cities['Category'] = "Cities"


In [53]:
data = pd.concat([cities,skills,qualifications], ignore_index=True)

In [54]:
data = data[['MSA','Category','Metric','Profiles']]

In [56]:
data.to_csv('deleteMeNow.csv', index=False)

In [55]:
data.head()

,MSA,Category,Metric,Profiles
0,Charlotte_Concord_Gastonia_NC_SC_3282,Cities,"Charlotte, NC",1719
1,Charlotte_Concord_Gastonia_NC_SC_3282,Cities,"Concord, NC",55
2,Charlotte_Concord_Gastonia_NC_SC_3282,Cities,"Fort Mill, SC",49
3,Charlotte_Concord_Gastonia_NC_SC_3282,Cities,"Huntersville, NC",40
4,Charlotte_Concord_Gastonia_NC_SC_3282,Cities,"Mooresville, NC",33


In [14]:
msaProfiles = pd.read_excel('deleteMsaProfiles.xlsx', usecols="B,C")

In [15]:
msaShape = gpd.read_file('../../../data/spatial/msa/us_msa.geojson')

In [16]:
msaProfiles = pd.merge(msaProfiles,msaShape,how='left',left_on='MSA',right_on='NAME')

In [17]:
msaProfiles = msaProfiles[['MSA','GEOID','Profiles','geometry']]

In [18]:
msaProfiles.sort_values('MSA', inplace=True)

In [20]:
crs = {'init': 'epsg:4326'}
msaProfiles = GeoDataFrame(msaProfiles, crs=crs, geometry=msaProfiles.geometry)

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [21]:
msaProfiles['geometry'] = msaProfiles['geometry'].centroid

In [26]:
msaProfiles.to_file('msaprofiles.geojson', driver="GeoJSON")

In [26]:
msaProfiles.head()

NameError: name 'msaProfiles' is not defined

In [28]:
data.to_excel('deleteMsaData.xlsx', index=False)

In [21]:
df = pd.read_excel('cyber-story-data.xlsx', sheet_name="msaProfilesData")

In [22]:
df = df.loc[df['Flagged'] == "Flagged"]

In [16]:
df['Rows'] = df['Rows'].round(0)

In [17]:
df['Rows'] = df['Rows'].astype(int)
df['Profiles'] = df['Profiles'].astype(int)

In [18]:
df = df[['MSA','Metric','Profiles','Rows']]

In [19]:
df.reset_index(drop=True,inplace=True)

In [22]:
data = df.loc[df.index.repeat(df.Rows)]

In [24]:
data.rename(columns={'Profiles':'Total'}, inplace=True)
data['Profiles'] = 5

In [2]:
totalProfiles = gpd.read_file('msaProfiles.geojson')

In [29]:
totalProfiles['Metric'] = "Total Cyber Profiles"

In [31]:
totalProfiles['Rows'] = totalProfiles['Profiles'] / 5
totalProfiles.rename(columns={'Profiles':'Total'}, inplace=True)

In [33]:
totalProfiles['Rows'] = totalProfiles['Rows'].round(0)
totalProfiles['Rows'] = totalProfiles['Rows'].astype(int)

In [35]:
data.head()

,MSA,Metric,Total,Rows,Profiles
0,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
0,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
0,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
0,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
0,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5


In [36]:
totalProfiles = totalProfiles[['MSA','Metric','Total','Rows']]

In [38]:
total = totalProfiles.loc[totalProfiles.index.repeat(totalProfiles.Rows)]

In [40]:
total['Profiles'] = 5

In [42]:
data.head(1)

,MSA,Metric,Total,Rows,Profiles
0,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5


In [41]:
total.head(1)

,MSA,Metric,Total,Rows,Profiles
0,"Atlanta-Sandy Springs-Alpharetta, GA",Total Cyber Profiles,8376,1675,5


In [45]:
df = pd.concat([data,total])
df.reset_index(drop=True,inplace=True)

In [46]:
df.head()

,MSA,Metric,Total,Rows,Profiles
0,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
1,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
2,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
3,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5
4,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346,69,5


In [48]:
df.to_csv('deleteMeNow.csv', index=False)

In [4]:
totalProfiles.to_csv('deleteMeNow.csv', index=False)

In [3]:
totalProfiles.head()

,MSA,GEOID,Profiles,geometry
0,"Atlanta-Sandy Springs-Alpharetta, GA",12060,8376,POINT (-84.39957 33.69277)
1,"Augusta-Richmond County, GA-SC",12260,1027,POINT (-81.98293 33.46104)
2,"Austin-Round Rock-Georgetown, TX",12420,2835,POINT (-97.65445 30.26261)
3,"Baltimore-Columbia-Towson, MD",12580,11442,POINT (-76.55101 39.30554)
4,"Boston-Cambridge-Newton, MA-NH",14460,7310,POINT (-71.02193 42.51790)


In [8]:
df = df[['MSA','Metric','Profiles']]
df['Profiles'] = df['Profiles'].astype(int)

In [10]:
df = df.pivot_table('Profiles', ['MSA'], 'Metric')

In [12]:
df.reset_index(inplace=True)

In [14]:
df.fillna(0, inplace=True)

In [18]:
cols = df.columns.drop('MSA')
df[cols] = df[cols].astype(int)

In [24]:
df = df[['MSA','Metric','Profiles']]
df['Profiles'] = df['Profiles'].astype(int)

In [25]:
df.head()

,MSA,Metric,Profiles
1052,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Ethical Hacker,346
1055,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Information Security Manager,300
1056,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Information System Auditor (CISA),252
1057,"Atlanta-Sandy Springs-Alpharetta, GA",Certified Information Systems Security Profess...,886
1067,"Atlanta-Sandy Springs-Alpharetta, GA",Cisco Certified Network Associate,449


In [26]:
df.to_csv('deleteMeNow.csv', index=False)